# 第四讲 功能扩展

## 一、 回顾

### 1、tf.keras 搭建神经网络八股——六步法

### 2、代码 mnist_train_baseline.py

## 二、本讲用 tf.keras 完善功能模块

### 1、自制数据集，应对特定应用

#### 1.1、观察数据集数据结构，配成特征标签对

### 2、数据增强，增大数据量

#### 2.1、数据增强（增大数据量）

#### 2.2、数据增强可视化 (代码 show_augmented _images.py)

### 3、断点续训，存取模型

#### 3.1、读取模型

#### 3.2、保存模型

### 4、参数提取，写至文本

#### 4.1、提取可训练参数

#### 4.2、设置 print 输出格式

### 5、acc/loss 可视化，查看效果

#### 5.1、acc 曲线与 loss 曲线

### 6、应用程序，给图识物

#### 6.1、给图识物

#### 6.2、前向传播执行应用

## 三、补充

### 1、数据集 Fashion_mnist